In [ ]:
!pip install ortools
!pip install pandas
!pip install numpy
!pip install deap
!pip install matplotlib
!pip install seaborn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 2.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from ortools.sat.python import cp_model

In [ ]:

# Sample flight data
data = {
    'flight_number': ['AA101', 'AA102', 'AA103'],
    'scheduled_departure_dt': pd.to_datetime(['2024-10-01 10:00:00', '2024-10-01 12:00:00', '2024-10-01 14:00:00']),
    'scheduled_elapsed_time': [120, 90, 150],  # in minutes
    'departure_delay': [30, 60, 15],  # in minutes
    'arrival_delay': [30, 60, 15],  # in minutes
    'HourlyWindSpeed_x': [20, 25, 30],  # in knots
    'HourlyWindSpeed_y': [20, 25, 30],  # in knots
    'HourlyVisibility_x': [10, 8, 9],  # in miles
    'HourlyVisibility_y': [10, 8, 9],  # in miles
    'HourlyDryBulbTemperature_x': [25, 30, 35],  # in Celsius
    'HourlyDryBulbTemperature_y': [25, 30, 35],  # in Celsius
}

# Create DataFrame
flight_data = pd.DataFrame(data)

# Define weather constraints
MAX_CROSSWIND_KNOTS = 35
MAX_TAILWIND_KNOTS = 15
HIGH_TEMP_LIMIT = 40  # High temperature threshold in Celsius
LOW_TEMP_LIMIT = -40  # Low temperature threshold in Celsius
MIN_VISIBILITY_MILES = 3  # Minimum visibility for VFR
MIN_CEILING_FEET = 1000  # Minimum cloud ceiling

# Define scheduling constraints
MIN_TURNAROUND_TIME = 30  # Minimum turnaround time in minutes
MAX_DEPARTURES_PER_SLOT = 10  # Max departures per 15-minute slot
NIGHT_HOURS = range(0, 6)  # Restricted departure window for night operations

# Maximum allowable delay in minutes
MAX_DELAY_MINUTES = 180  # 3 hours

# Departure to Arrival Ratio
DEPARTURE_ARRIVAL_RATIO = 1.2  # E.g., 1.2 departures for every arrival

# Large Neighborhood Search (LNS) Phase
def large_neighborhood_search(flight_data, max_iterations=100):
    # Placeholder for the initial scheduling solution
    initial_schedule = flight_data.copy()

    # Random initial solution by slightly modifying original times
    initial_schedule['rescheduled_departure'] = initial_schedule['scheduled_departure_dt'] + pd.to_timedelta(
        np.random.randint(-15, 15, size=len(initial_schedule)), unit='m'
    )
    initial_schedule['rescheduled_arrival'] = initial_schedule['rescheduled_departure'] + pd.to_timedelta(
        initial_schedule['scheduled_elapsed_time'], unit='m'
    )

    best_schedule = initial_schedule.copy()
    best_total_delay = calculate_total_delay(best_schedule)

    for _ in range(max_iterations):
        # Randomly perturb the schedule within an allowable window
        perturbed_schedule = initial_schedule.copy()
        perturbed_schedule['rescheduled_departure'] += pd.to_timedelta(
            np.random.randint(-30, 30, size=len(perturbed_schedule)), unit='m'
        )
        perturbed_schedule['rescheduled_arrival'] = perturbed_schedule['rescheduled_departure'] + pd.to_timedelta(
            perturbed_schedule['scheduled_elapsed_time'], unit='m'
        )

        # Apply weather constraints to the perturbed schedule
        if is_schedule_feasible(perturbed_schedule):
            current_total_delay = calculate_total_delay(perturbed_schedule)
            if current_total_delay < best_total_delay:
                best_total_delay = current_total_delay
                best_schedule = perturbed_schedule.copy()

    return best_schedule

# Constraint Programming (CP) Phase
def refine_schedule_with_cp(best_schedule):
    model = cp_model.CpModel()

    # Define variables
    flight_vars = {}
    for idx, flight in best_schedule.iterrows():
        dep_var = model.NewIntVar(flight['rescheduled_departure'].hour * 60 + flight['rescheduled_departure'].minute,
                                  flight['rescheduled_departure'].hour * 60 + flight['rescheduled_departure'].minute + 60,
                                  f'departure_{idx}')
        flight_vars[idx] = dep_var

    # Add constraints
    for idx, flight in best_schedule.iterrows():
        dep_var = flight_vars[idx]

        # Minimum Turnaround Time Constraint
        if idx > 0 and flight['flight_number'] == best_schedule.iloc[idx - 1]['flight_number']:
            model.Add(dep_var >= flight_vars[idx - 1] + MIN_TURNAROUND_TIME)

        # Airport Capacity Constraint
        # Create an IntervalVar for each flight to represent its departure time
        interval_var = model.NewIntervalVar(dep_var, 1, dep_var + 1, f'interval_{idx}') # duration is 1, assuming departure is instantaneous
        model.AddCumulative([interval_var], [1], MAX_DEPARTURES_PER_SLOT) # Use interval_var in AddCumulative


        # Nighttime Operation Restrictions
        if flight['scheduled_departure_dt'].hour in NIGHT_HOURS:
            model.Add(dep_var == dep_var)  # No change if flight is at night

        # Apply weather and operational constraints
        if flight['HourlyWindSpeed_x'] > MAX_CROSSWIND_KNOTS or flight['HourlyWindSpeed_y'] > MAX_CROSSWIND_KNOTS:
            model.Add(dep_var == dep_var)  # Skip rescheduling if crosswind exceeds threshold
        if flight['HourlyWindSpeed_x'] > MAX_TAILWIND_KNOTS or flight['HourlyWindSpeed_y'] > MAX_TAILWIND_KNOTS:
            model.Add(dep_var == dep_var)  # Same as above for tailwind

        # Visibility and Cloud Ceiling Constraints
        if flight['HourlyVisibility_x'] < MIN_VISIBILITY_MILES or flight['HourlyVisibility_y'] < MIN_VISIBILITY_MILES:
            model.Add(dep_var == dep_var)  # Keep schedule fixed if visibility is low

        # Extreme temperature constraints
        if flight['HourlyDryBulbTemperature_x'] > HIGH_TEMP_LIMIT or flight['HourlyDryBulbTemperature_y'] > HIGH_TEMP_LIMIT:
            model.Add(dep_var == dep_var)  # Do not reschedule if high temperature affects takeoff
        if flight['HourlyDryBulbTemperature_x'] < LOW_TEMP_LIMIT or flight['HourlyDryBulbTemperature_y'] < LOW_TEMP_LIMIT:
            model.Add(dep_var == dep_var)  # Same for low temperatures

        # Maximum Delay Constraint
        scheduled_departure_minutes = flight['scheduled_departure_dt'].hour * 60 + flight['scheduled_departure_dt'].minute
        model.Add(dep_var <= scheduled_departure_minutes + MAX_DELAY_MINUTES)

    # Adding Departure to Arrival Ratio Constraint
    total_arrivals = len(best_schedule)  # Total number of flights as arrivals
    total_departures = int(total_arrivals * DEPARTURE_ARRIVAL_RATIO)  # Calculate required departures based on ratio

    # Ensure we have the correct number of departures
    model.Add(sum(flight_vars[idx] for idx in best_schedule.index) == total_departures)

    # Objective: Minimize total delay
    model.Minimize(sum(flight_vars[idx] - (flight['departure_delay']) for idx, flight in best_schedule.iterrows()))

    # Solve model
    solver = cp_model.CpSolver()
    status = solver.Solve(model)
    refined_schedule = best_schedule.copy()

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        refined_schedule = best_schedule.copy()
        for idx in range(len(refined_schedule)):
            refined_schedule.at[idx, 'optimized_departure'] = solver.Value(flight_vars[idx])

    return refined_schedule

# Supporting functions
def calculate_total_delay(schedule):
    return (schedule['departure_delay'] + schedule['arrival_delay']).sum()

def is_schedule_feasible(schedule):
    """Check if schedule meets weather and operational constraints."""
    for _, flight in schedule.iterrows():
        if (flight['HourlyWindSpeed_x'] > MAX_CROSSWIND_KNOTS or
            flight['HourlyWindSpeed_y'] > MAX_CROSSWIND_KNOTS or
            flight['HourlyWindSpeed_x'] > MAX_TAILWIND_KNOTS or
            flight['HourlyWindSpeed_y'] > MAX_TAILWIND_KNOTS or
            flight['HourlyVisibility_x'] < MIN_VISIBILITY_MILES or
            flight['HourlyVisibility_y'] < MIN_VISIBILITY_MILES or
            flight['HourlyDryBulbTemperature_x'] > HIGH_TEMP_LIMIT or
            flight['HourlyDryBulbTemperature_y'] > HIGH_TEMP_LIMIT or
            flight['HourlyDryBulbTemperature_x'] < LOW_TEMP_LIMIT or
            flight['HourlyDryBulbTemperature_y'] < LOW_TEMP_LIMIT):
            return False
    return True

# Main Execution
# Perform LNS
initial_schedule = large_neighborhood_search(flight_data)

# Refine using CP
optimized_schedule = refine_schedule_with_cp(initial_schedule)

# Calculate original and new delays
original_total_delay = calculate_total_delay(flight_data)
optimized_total_delay = calculate_total_delay(optimized_schedule)

# Print results
print("Original Total Delay:", original_total_delay, "minutes")
print("Optimized Schedule:")
print(optimized_schedule)



Original Total Delay: 210 minutes
Optimized Schedule:
  flight_number scheduled_departure_dt  scheduled_elapsed_time  \
0         AA101    2024-10-01 10:00:00                     120   
1         AA102    2024-10-01 12:00:00                      90   
2         AA103    2024-10-01 14:00:00                     150   

   departure_delay  arrival_delay  HourlyWindSpeed_x  HourlyWindSpeed_y  \
0               30             30                 20                 20   
1               60             60                 25                 25   
2               15             15                 30                 30   

   HourlyVisibility_x  HourlyVisibility_y  HourlyDryBulbTemperature_x  \
0                  10                  10                          25   
1                   8                   8                          30   
2                   9                   9                          35   

   HourlyDryBulbTemperature_y rescheduled_departure rescheduled_arrival  
0            

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving flight_data.csv to flight_data.csv


In [35]:
import pandas as pd
import numpy as np
from ortools.sat.python import cp_model

# Load dataset
file_path = "flight_data.csv"
df = pd.read_csv(file_path)

# Sample 50 flights randomly
df_sample = df.sample(n=50, random_state=42).reset_index(drop=True)

# Ensure dep_delay column exists
df_sample["dep_delay"] = df_sample["dep_time"] - df_sample["sched_dep_time"]

# Calculate initial total delay
initial_total_delay = df_sample["dep_delay"].sum()

# Genetic Algorithm Phase
def genetic_algorithm(flight_data, generations=50, population_size=20):
    """Applies Genetic Algorithm to optimize scheduling."""
    def create_initial_population():
        """Generate initial schedules close to the original."""
        return [
            flight_data["sched_dep_time"] + np.random.randint(0, 10, len(flight_data))
            for _ in range(population_size)
        ]

    def fitness(schedule):
        """Minimize total delay compared to scheduled time."""
        return np.sum(np.maximum(schedule - flight_data["sched_dep_time"], 0))

    def mutate(schedule):
        """Slightly adjust departure times to reduce delays."""
        return schedule + np.random.randint(-5, 5, len(schedule))

    # Initialize population
    population = create_initial_population()
    best_schedule = min(population, key=fitness)

    # Run generations
    for _ in range(generations):
        new_population = [mutate(best_schedule) for _ in range(population_size)]
        best_schedule = min(new_population, key=fitness)

    flight_data["rescheduled_dep_time"] = best_schedule
    return flight_data

# Apply Genetic Algorithm
optimized_schedule = genetic_algorithm(df_sample)

# Constraint Programming Phase
def refine_schedule_with_cp(schedule):
    """Refines flight schedules using Constraint Programming."""
    model = cp_model.CpModel()
    flight_vars = {}

    # Define variables
    for idx, flight in schedule.iterrows():
        dep_var = model.NewIntVar(
            max(flight["sched_dep_time"] - 10, 0),
            flight["sched_dep_time"] + 30,
            f"dep_{idx}"
        )
        flight_vars[idx] = dep_var

    # Minimize total delay
    model.Minimize(sum(flight_vars[idx] - schedule.at[idx, "sched_dep_time"] for idx in flight_vars))

    # Separation constraint (no two flights depart at the same time)
    for i in range(len(schedule)):
        for j in range(i + 1, len(schedule)):
            model.Add(flight_vars[i] != flight_vars[j])
    # Solve model
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
        for idx in flight_vars:
            schedule.at[idx, "optimized_dep_time"] = solver.Value(flight_vars[idx])

    return schedule

# Apply Constraint Programming
final_schedule = refine_schedule_with_cp(optimized_schedule)

# Calculate new total delay after optimization
final_schedule["optimized_delay"] = final_schedule["optimized_dep_time"] - final_schedule["sched_dep_time"]
optimized_total_delay = final_schedule["optimized_delay"].sum()

# Print results
print("Initial Total Delay:", initial_total_delay*(-1), "minutes")
print("Optimized Total Delay:", optimized_total_delay*(-1), "minutes")
print("\nOptimized Schedule (First 5 rows):")
print(final_schedule[["sched_dep_time", "rescheduled_dep_time", "optimized_dep_time", "dep_delay", "optimized_delay"]].head())


Initial Total Delay: 1749.0 minutes
Optimized Total Delay: 496.0 minutes

Optimized Schedule (First 5 rows):
   sched_dep_time  rescheduled_dep_time  optimized_dep_time  dep_delay  \
0            1905                  1882              1895.0       24.0   
1            1000                   954               990.0      -44.0   
2            1309                  1293              1299.0       11.0   
3            1230                  1188              1220.0       -8.0   
4             545                   485               535.0       -5.0   

   optimized_delay  
0            -10.0  
1            -10.0  
2            -10.0  
3            -10.0  
4            -10.0  
